In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.5
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [ ]:
import tensorflow as tf
print(tf.__version__)

2.12.0


In [ ]:
# import the libraries as shown below

from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob
#import matplotlib.pyplot as plt

In [ ]:
!pip install opendatasets
!pip install pandas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# re-size all the images to this
IMAGE_SIZE = [224, 224]

train_path = '/content/drive/MyDrive/train'
valid_path = '/content/drive/MyDrive/test'

In [ ]:
vgg16 = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

In [ ]:
# don't train existing weights
for layer in vgg16.layers:
    layer.trainable = False

In [ ]:
# our layers - you can add more if you want
x = Flatten()(vgg16.output)

In [ ]:
folders = glob('/content/drive/MyDrive/train/*')

In [ ]:
x = Flatten()(vgg16.output)


In [ ]:

len(folders)

2

In [ ]:
prediction = Dense(len(folders), activation='softmax')(x)

# create a model object
model = Model(inputs=vgg16.input, outputs=prediction)

In [ ]:
# view the structure of the model
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0   

In [ ]:
# tell the model what cost and optimization method to use
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)


In [ ]:
# Use the Image Data Generator to import the images from the dataset
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [ ]:
# Make sure you provide the same target size as initialied for the image size
training_set = train_datagen.flow_from_directory('/content/drive/MyDrive/train',target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical'
                                                 )

Found 9793 images belonging to 2 classes.


In [1]:
test_set = test_datagen.flow_from_directory('/content/drive/MyDrive/test',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

NameError: name 'test_datagen' is not defined

In [ ]:
# fit the model
# Run the cell. It will take some time to execute
r = model.fit(
  training_set,
  validation_data=test_set,
  epochs=5,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

Epoch 1/5
188/307 [=================>............] - ETA: 37:44 - loss: 0.6443 - accuracy: 0.7113

In [ ]:
r.history

{'loss': [0.7891513705253601,
  1.0237971544265747,
  0.9041279554367065,
  0.7277421355247498,
  0.4868132174015045,
  0.5793066024780273,
  0.6007969379425049,
  0.6025451421737671,
  0.7105629444122314,
  0.3957803547382355,
  0.32524773478507996,
  0.27742326259613037,
  0.29896488785743713,
  0.2962772250175476,
  0.2811773121356964,
  0.2063586413860321,
  0.2696796655654907,
  0.25683215260505676,
  0.226004958152771,
  0.19163787364959717],
 'accuracy': [0.37037035822868347,
  0.7407407164573669,
  0.7407407164573669,
  0.7407407164573669,
  0.8148148059844971,
  0.8148148059844971,
  0.8148148059844971,
  0.8148148059844971,
  0.8148148059844971,
  0.8518518805503845,
  0.8518518805503845,
  0.8888888955116272,
  0.8518518805503845,
  0.8518518805503845,
  0.8148148059844971,
  0.9629629850387573,
  0.8888888955116272,
  0.9629629850387573,
  1.0,
  1.0],
 'val_loss': [1.583293080329895,
  1.1132563352584839,
  0.5288726091384888,
  0.7048142552375793,
  1.0816603899002075,
  

In [ ]:
result = model.evaluate(test_set)
dict(zip(model.metrics_names, result))

1/1 [==============================] - 10s 10s/step - loss: 0.3204 - accuracy: 0.9286


{'loss': 0.3203723132610321, 'accuracy': 0.9285714030265808}

In [ ]:
result2 = model.evaluate(training_set)
dict(zip(model.metrics_names, result2))

1/1 [==============================] - 17s 17s/step - loss: 0.1705 - accuracy: 0.9630


{'loss': 0.17052382230758667, 'accuracy': 0.9629629850387573}

In [ ]:
#Training accuracy is 96% while test accuracy is 92%